In [1]:
%run ../src/ipyautoui/__init__.py
%load_ext lab_black

In [19]:
print(" not yet implemented".upper())

 NOT YET IMPLEMENTED


# Validation - NOTE:  NOT YET IMPLEMENTED

- jsonschema validation occurs on-change by default (TODO: implement!)
    - review https://github.com/pydantic/pydantic-core
- the top-level pydantic model can also be validated on request (slows things down...?)
    - possible to implement recursive validation with pydantic? 

In [3]:
from jsonschema import validate, Draft202012Validator, ErrorTree

# A sample schema, like what we'd get from json.load()
schema = {
    "type" : "object",
    "properties" : {
        "price" : {"type" : "number"},
        "name" : {"type" : "string"},
    },
}

# If no exception is raised by validate(), the instance is valid.
validate(instance={"name" : "Eggs", "price" : 34.99}, schema=schema)

validate(
    instance={"name" : "Eggs", "price" : "Invalid"}, schema=schema,
)      

ValidationError: 'Invalid' is not of type 'number'

Failed validating 'type' in schema['properties']['price']:
    {'type': 'number'}

On instance['price']:
    'Invalid'

In [4]:
schema = {
    "items": {
        "anyOf": [{"type": "string", "maxLength": 2}, {"type": "integer", "minimum": 5}]
    }
}
instance = [{}, 3, "foo"]
v = Draft202012Validator(schema)
errors = sorted(v.iter_errors(instance), key=lambda e: e.path)

In [6]:
from ipyautoui.constants import TRUE_BUTTON_KWARGS, BUTTON_WIDTH_MIN, BUTTON_HEIGHT_MIN
import ipywidgets as w

BLANK_BUTTON_KWARGS = dict(
    icon="times",
    style={"button_color": "red"},
    layout={"width": BUTTON_WIDTH_MIN, "height": BUTTON_HEIGHT_MIN},
    disabled=True,
)

BLANK_BUTTON_KWARGS = dict(
    icon="check",
    style={"button_color": "white"},
    layout={"width": BUTTON_WIDTH_MIN, "height": BUTTON_HEIGHT_MIN},
    disabled=True,
)
w.HBox([w.Button(**BLANK_BUTTON_KWARGS), w.Text("asdfasdf")])

In [7]:
for error in errors:
    for suberror in sorted(error.context, key=lambda e: e.schema_path):
        print(list(suberror.schema_path), suberror.message, sep=", ")

[0, 'type'], {} is not of type 'string'
[1, 'type'], {} is not of type 'integer'
[0, 'type'], 3 is not of type 'string'
[1, 'minimum'], 3 is less than the minimum of 5
[0, 'maxLength'], 'foo' is too long
[1, 'type'], 'foo' is not of type 'integer'


In [8]:
schema = {
    "type": "array",
    "items": {"type": "number", "enum": [1, 2, 3]},
    "minItems": 3,
}
instance = ["spam", 2]

v = Draft202012Validator(schema)
for error in sorted(v.iter_errors(["spam", 2]), key=str):
    print(error.message)

'spam' is not of type 'number'
'spam' is not one of [1, 2, 3]
['spam', 2] is too short


In [9]:
tree = ErrorTree(v.iter_errors(instance))

In [17]:
# REF: https://docs.pydantic.dev/latest/concepts/validators/#model-validators

from typing import Any
from pydantic import BaseModel, ValidationError, model_validator


class UserModel(BaseModel):
    username: str
    password1: str
    password2: str

    @model_validator(mode="before")
    @classmethod
    def check_card_number_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            assert "card_number" not in data, "card_number should not be included"
        return data

    @model_validator(mode="after")
    def check_passwords_match(self) -> "UserModel":
        pw1 = self.password1
        pw2 = self.password2
        if pw1 is not None and pw2 is not None and pw1 != pw2:
            raise ValueError("passwords do not match")
        return self


print(UserModel(username="scolvin", password1="zxcvbn", password2="zxcvbn"))
# > username='scolvin' password1='zxcvbn' password2='zxcvbn'
try:
    UserModel(username="scolvin", password1="zxcvbn", password2="zxcvbn2")
except ValidationError as e:
    print(e)
    """
    1 validation error for UserModel
      Value error, passwords do not match [type=value_error, input_value={'username': 'scolvin', '... 'password2': 'zxcvbn2'}, input_type=dict]
    """

try:
    UserModel(
        username="scolvin",
        password1="zxcvbn",
        password2="zxcvbn",
        card_number="1234",
    )
except ValidationError as e:
    print(e)
    """
    1 validation error for UserModel
      Assertion failed, card_number should not be included
    assert 'card_number' not in {'card_number': '1234', 'password1': 'zxcvbn', 'password2': 'zxcvbn', 'username': 'scolvin'} [type=assertion_error, input_value={'username': 'scolvin', '..., 'card_number': '1234'}, input_type=dict]
    """

username='scolvin' password1='zxcvbn' password2='zxcvbn'
1 validation error for UserModel
  Value error, passwords do not match [type=value_error, input_value={'username': 'scolvin', '... 'password2': 'zxcvbn2'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1/v/value_error
1 validation error for UserModel
  Assertion failed, card_number should not be included [type=assertion_error, input_value={'username': 'scolvin', '..., 'card_number': '1234'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1/v/assertion_error


In [18]:
import stringcase as sc

name = w.Text()
name_ = w.Text(disabled=True)
vbox = w.VBox(
    [
        w.HBox([name, w.Label("schedule name in plain english (with spaces)")]),
        w.HBox([name_, w.Label("schedule name for machines")]),
    ]
)


def on_name_change(on_change):
    name_.value = sc.pascalcase(sc.snakecase(name.value.lower())).strip("_")


name.observe(on_name_change, "value")

vbox